IMPORTS

In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

Matplotlib is building the font cache; this may take a moment.


In [69]:
df = pd.read_csv('data/Combined-spreadsheet-V4-copy.csv', header=2)
df.head()

,Notes,Patient,Centre,Visit Date,Visit date in relation to BVP,Year of BVP,Date of BVP,Date of birth,Age at BVP (days),Age at BVP (months),...,Outcome,50% reduction,Important Notes,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82
0,NaN,1.0,1.0,21/07/2022,eg - 50 days,NaN,NaN,12/05/2022,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Restenosis - Does PG at the 4th row increase b...
1,NaN,NaN,NaN,04/08/2022,e.g. 0 days,NaN,04/08/2022,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,05/08/2022,1 day,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Outcome - compare 4th row to ideally 2nd row, ..."
3,NaN,NaN,NaN,06/09/2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,17/02/2023,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50% reduction - does 4th row PG reduce by >/in...


In [70]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [71]:
# forward fill our patient ID
df['Patient'] = df['Patient'].ffill()

In [72]:
# group by patient and fill columns that have single values with the values
df['Centre'] = df.groupby('Patient')['Centre'].transform('first')
df['Date of BVP'] = df.groupby('Patient')['Date of BVP'].transform('first')
df['Date of birth'] = df.groupby('Patient')['Date of birth'].transform('first')
df['Breed'] = df.groupby('Patient')['Breed'].transform('first')
df['Brachycephalic (Y/N)'] = df.groupby('Patient')['Brachycephalic (Y/N)'].transform('first')
df['Sex'] = df.groupby('Patient')['Sex'].transform('first')
df['PS Type (A/B)'] = df.groupby('Patient')['PS Type (A/B)'].transform('first')
df['Leaflet Thickening (Y/N)'] = df.groupby('Patient')['Leaflet Thickening (Y/N)'].transform('first')
df['Annular Hypoplasia (Y/N)'] = df.groupby('Patient')['Annular Hypoplasia (Y/N)'].transform('first')
df['Fibrotic tissue at annulus (Y/N)'] = df.groupby('Patient')['Fibrotic tissue at annulus (Y/N)'].transform('first')
df['Fibrotic tissue at sinotubular junction (Y/N)'] = df.groupby('Patient')['Fibrotic tissue at sinotubular junction (Y/N)'].transform('first')
df['Post-stenotic dilation (N/Mild/Moderate/Severe)'] = df.groupby('Patient')['Post-stenotic dilation (N/Mild/Moderate/Severe)'].transform('first')
df['Pulmonary Annulus (mm)'] = df.groupby('Patient')['Pulmonary Annulus (mm)'].transform('first')
df['Aortic Annulus (mm)'] = df.groupby('Patient')['Aortic Annulus (mm)'].transform('first')
df['Ao/PV ratio'] = df.groupby('Patient')['Ao/PV ratio'].transform('first')
df['Pulmonary Annulus (angio)'] = df.groupby('Patient')['Pulmonary Annulus (angio)'].transform('first')
df['Balloon Type/Brand'] = df.groupby('Patient')['Balloon Type/Brand'].transform('first')
df['High/Low Pressure Balloon'] = df.groupby('Patient')['High/Low Pressure Balloon'].transform('first')
df['Balloon Width (mm)'] = df.groupby('Patient')['Balloon Width (mm)'].transform('first')
df['Smaller balloons used'] = df.groupby('Patient')['Smaller balloons used'].transform('first')
df['Balloon Length (mm)'] = df.groupby('Patient')['Balloon Length (mm)'].transform('first')
df['Balloon:annulus (angio)'] = df.groupby('Patient')['Balloon:annulus (angio)'].transform('first')
df['Date of death'] = df.groupby('Patient')['Date of death'].transform('first')
df['Date last known alive'] = df.groupby('Patient')['Date last known alive'].transform('first')
df['Onset of clinical signs'] = df.groupby('Patient')['Onset of clinical signs'].transform('first')
df['Comments'] = df.groupby('Patient')['Comments'].transform('first')
df['Important Notes'] = df.groupby('Patient')['Important Notes'].transform('first')

In [73]:
# fill brachycephalic with N if NaN
df['Brachycephalic (Y/N)'] = df['Brachycephalic (Y/N)'].fillna('N')
df['Brachycephalic (Y/N)'].value_counts()

Brachycephalic (Y/N)
N    735
Y    367
Name: count, dtype: int64

In [74]:
# check for empty visit dates
df['Visit Date'].isna().sum()

np.int64(62)

In [75]:
# Adding some visit number / preop postop fields
df['Visit_Number'] = df.groupby('Patient').cumcount() + 1

def assign_visit_type(visit_num):
    if visit_num == 1:
        return 'Preop_1'
    elif visit_num == 2:
        return 'Preop_2'
    elif visit_num == 3:
        return 'Postop_1'
    elif visit_num == 4:
        return 'Postop_2'
    else:
        return f'Postop_{visit_num - 2}'  # For additional postop visits

df['Visit_Type'] = df['Visit_Number'].apply(assign_visit_type)

In [76]:
df['Visit Date'] = pd.to_datetime(df['Visit Date'], format='%d/%m/%Y', errors='coerce')
df['Date of BVP'] = pd.to_datetime(df['Date of BVP'], format='%d/%m/%Y', errors='coerce')

# Calculate days from bvp
df['Visit date in relation to BVP'] = (df['Visit Date'] - df['Date of BVP']).dt.days

# Add year col
df['Year of BVP'] = df['Date of BVP'].dt.year

df[['Patient', 'Visit_Number', 'Visit_Type', 'Visit Date', 'Date of BVP', 'Visit date in relation to BVP', 'Year of BVP']].head(10)

,Patient,Visit_Number,Visit_Type,Visit Date,Date of BVP,Visit date in relation to BVP,Year of BVP
0,1.0,1,Preop_1,2022-07-21,2022-08-04,-14.0,2022
1,1.0,2,Preop_2,2022-08-04,2022-08-04,0.0,2022
2,1.0,3,Postop_1,2022-08-05,2022-08-04,1.0,2022
3,1.0,4,Postop_2,2022-09-06,2022-08-04,33.0,2022
4,1.0,5,Postop_3,2023-02-17,2022-08-04,197.0,2022
5,1.0,6,Postop_4,2023-10-06,2022-08-04,428.0,2022
6,2.0,1,Preop_1,2021-09-24,2021-10-22,-28.0,2021
7,2.0,2,Preop_2,2021-10-21,2021-10-22,-1.0,2021
8,2.0,3,Postop_1,2021-10-22,2021-10-22,0.0,2021
9,2.0,4,Postop_2,2021-11-26,2021-10-22,35.0,2021


In [77]:
# adding patient age at BVP
df['Date of birth'] = pd.to_datetime(df['Date of birth'], format='%d/%m/%Y', errors='coerce')

df['Age at BVP (days)'] = (df['Date of BVP'] - df['Date of birth']).dt.days

# months calculation - how do you want this??
df['Age at BVP (months)'] = round(df['Age at BVP (days)'] / 30.44)

df[['Patient', 'Date of birth', 'Date of BVP', 'Age at BVP (days)', 'Age at BVP (months)']].head(10)

,Patient,Date of birth,Date of BVP,Age at BVP (days),Age at BVP (months)
0,1.0,2022-05-12,2022-08-04,84.0,3.0
1,1.0,2022-05-12,2022-08-04,84.0,3.0
2,1.0,2022-05-12,2022-08-04,84.0,3.0
3,1.0,2022-05-12,2022-08-04,84.0,3.0
4,1.0,2022-05-12,2022-08-04,84.0,3.0
5,1.0,2022-05-12,2022-08-04,84.0,3.0
6,2.0,2021-05-24,2021-10-22,151.0,5.0
7,2.0,2021-05-24,2021-10-22,151.0,5.0
8,2.0,2021-05-24,2021-10-22,151.0,5.0
9,2.0,2021-05-24,2021-10-22,151.0,5.0


In [81]:
# Atenolol start date - visit date
# first check for instances where there is no corresponding visit date
med_no_visit = df[df['Atenolol start date'].notna() & df['Visit Date'].isna()]
print(f"Number of medication starts without visit dates: {len(med_no_visit)}")
med_no_visit[['Patient', 'Visit_Number', 'Atenolol start date', 'Visit Date']]

Number of medication starts without visit dates: 7


,Patient,Visit_Number,Atenolol start date,Visit Date
496,88.0,2,17/11/2011,NaT
738,133.0,2,17/07/2023,NaT
827,149.0,2,09/04/2008,NaT
832,150.0,2,10/03/2010,NaT
862,156.0,2,19/08/2009,NaT
880,159.0,2,26/06/2014,NaT
936,169.0,2,23/10/2015,NaT


In [ ]:
# TODO: at the moment, I am going to use the previous visit in cases where there is no matching visit date
# TODO: check the order of the operation (Atenolol date - visit)

df['Atenolol start date'] = pd.to_datetime(df['Atenolol start date'], format='%d/%m/%Y', errors='coerce')
df['Visit Date for Atenolol Calc'] = pd.to_datetime(df['Visit Date for Atenolol Calc'], format='%d/%m/%Y', errors='coerce')

df['Previous Visit Date'] = df.groupby('Patient')['Visit Date'].shift(1)

# Use current visit date if available, otherwise use previous
df['Visit Date for Atenolol Calc'] = df['Visit Date'].fillna(df['Previous Visit Date'])

df['Days Atenolol Start to Visit'] = (df['Atenolol start date'] - df['Visit Date for Atenolol Calc']).dt.days

used_previous = df[df['Atenolol start date'].notna() & df['Visit Date'].isna() & df['Previous Visit Date'].notna()]
print(f"Cases using previous visit date: {len(used_previous)}")

no_visit_at_all = df[df['Atenolol start date'].notna() & df['Visit Date for Atenolol Calc'].isna()]
print(f"Cases with no visit date available: {len(no_visit_at_all)}")

df[['Patient', 'Visit_Number', 'Visit Date', 'Atenolol start date', 'Days Atenolol Start to Visit']].head(10)

Cases using previous visit date: 7
Cases with no visit date available: 0


,Patient,Visit_Number,Visit Date,Atenolol start date,Days Atenolol Start to Visit
0,1.0,1,2022-07-21,NaT,NaN
1,1.0,2,2022-08-04,2022-07-21,-14.0
2,1.0,3,2022-08-05,NaT,NaN
3,1.0,4,2022-09-06,NaT,NaN
4,1.0,5,2023-02-17,NaT,NaN
5,1.0,6,2023-10-06,NaT,NaN
6,2.0,1,2021-09-24,NaT,NaN
7,2.0,2,2021-10-21,2021-09-24,-27.0
8,2.0,3,2021-10-22,NaT,NaN
9,2.0,4,2021-11-26,NaT,NaN


In [86]:
df['Atenolol start date'] = df.groupby('Patient')['Atenolol start date'].transform('first')
df['Days Atenolol Start to Visit'] = df.groupby('Patient')['Days Atenolol Start to Visit'].transform('first')
df[['Patient', 'Visit_Number', 'Visit Date', 'Atenolol start date', 'Days Atenolol Start to Visit']].head(10)

,Patient,Visit_Number,Visit Date,Atenolol start date,Days Atenolol Start to Visit
0,1.0,1,2022-07-21,2022-07-21,-14.0
1,1.0,2,2022-08-04,2022-07-21,-14.0
2,1.0,3,2022-08-05,2022-07-21,-14.0
3,1.0,4,2022-09-06,2022-07-21,-14.0
4,1.0,5,2023-02-17,2022-07-21,-14.0
5,1.0,6,2023-10-06,2022-07-21,-14.0
6,2.0,1,2021-09-24,2021-09-24,-27.0
7,2.0,2,2021-10-21,2021-09-24,-27.0
8,2.0,3,2021-10-22,2021-09-24,-27.0
9,2.0,4,2021-11-26,2021-09-24,-27.0


In [ ]:
# Days after BVP Death (what about NaN)
# Days after BVP last known alive (what about NaN) - use last appt?